In [103]:
my_key = "LWzIQxzkc%2BeUq%2BQtoLVzpRCRy%2FvjodN7HU%2FY2NH6W%2Fj87kqP4avqY0Qfokhzd0dppcovgL%2BldJS398wibMSCXQ%3D%3D"

In [95]:
# my_key = "0C6iX2Ww%2BnB9GVY6k9Et5H9zqv%2FxjEsldVq1Wa9Co3lvcuaRi%2BI3gcoFoi3AckHg6B8AAf1cH0tKH4MXhYf57A%3D%3D"

In [104]:
end_point = "http://openapi.tour.go.kr/openapi/service/"

In [105]:
op_url = "EdrcntTourismStatsService/getEdrcntTourismStatsList?"
#YM=201201&NAT_CD=100&ED_CD=D&serviceKey=

In [106]:
ym = "202002" ; nat_cd = "112" ; ed_cd = "E"
param_ym = "&YM=" + ym
param_nat_cd = "&NAT_CD=" + nat_cd
param_ed_cd = "&ED_CD=" + ed_cd
param_key = "_type=json&serviceKey=" + my_key

In [107]:
url = end_point + op_url + param_key + param_ym + param_nat_cd + param_ed_cd
print(url)

http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList?_type=json&serviceKey=LWzIQxzkc%2BeUq%2BQtoLVzpRCRy%2FvjodN7HU%2FY2NH6W%2Fj87kqP4avqY0Qfokhzd0dppcovgL%2BldJS398wibMSCXQ%3D%3D&YM=202002&NAT_CD=112&ED_CD=E


### DataFrame 만들기

In [6]:
import pandas as pd
import urllib.request
import json

In [23]:
res = urllib.request.urlopen(url)

In [24]:
json_str = res.read().decode("utf-8"); json_str

'{"response":{"header":{"resultCode":"0000","resultMsg":"OK"},"body":{"items":{"item":{"ed":"방한외래관광객","edCd":"E","natCd":112,"natKorNm":"중  국","num":104086,"rnum":1,"ym":202002}},"numOfRows":10,"pageNo":1,"totalCount":1}}}'

In [25]:
json_object = json.loads(json_str)

In [26]:
pd.DataFrame(json_object["response"]["body"]["items"])

,item
ed,방한외래관광객
edCd,E
natCd,112
natKorNm,중 국
num,104086
rnum,1
ym,202002


In [27]:
pd.json_normalize(json_object['response']['body']['items'])

,item.ed,item.edCd,item.natCd,item.natKorNm,item.num,item.rnum,item.ym
0,방한외래관광객,E,112,중 국,104086,1,202002


### 출입국 데이터 만들기
1. 국가 3개 -> 112(중국), 130(일본), 275(미국)
2. 기간 -> 2019, 2020 년도
    - 연도별 01월 ~ 12월
3. 국가별, 연월별 방문객수 구하기
    - ["국가명", "년월", "방문객수"]
    - 총 데이터 개수 : 72개

In [28]:
# 4. 작업방법
#     - url을 받아서 item을 return 해주는 함수 만들기
#     - for 국가별
#     -     for 연도
#     -         for 월
#     -            url 조합 후 함수 실행
#     -            return 받은 데이터 -> DataFram에 합치기

In [108]:
import requests
def get_item(url, params):
    req = requests.get(url, params)
    item = req.json()["response"]["body"]["items"]
    return item

In [109]:
param_ym1 = [2019,2020]
param_nat_cd1 = ["112", "130", "275"]

In [110]:
op_url1 = "EdrcntTourismStatsService/getOvseaTuristStatsList?"

nit = 0

for nat in param_nat_cd1:
    for year in param_ym1:
        for month in range(1,13):
            params = {"YM":str(year) + "%02d"%month,
                     "NAT_CD" : nat,
                      "ED_DE" : "E"
                     }
            url1 = end_point + op_url1 + param_key
            print(url) 
            result = get_item(url, params)
            df = pd.json_normalize(result)
            if init == 0:
                df_total = df.copy(); init = 1
            else:
                df_total = pd.concat([df_total, df])
df_total

http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList?_type=json&serviceKey=LWzIQxzkc%2BeUq%2BQtoLVzpRCRy%2FvjodN7HU%2FY2NH6W%2Fj87kqP4avqY0Qfokhzd0dppcovgL%2BldJS398wibMSCXQ%3D%3D&YM=202002&NAT_CD=112&ED_CD=E


KeyError: 'body'